Task 5: A/B-тестирование

In [1]:
import pandas as pd

In [2]:
from sqlite3 import connect

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Создайте соединение с базой данных с помощью библиотеки sqlite3.

In [4]:
con = connect('/content/drive/MyDrive/School21/day11/data/checking-logs.sqlite')

Используя только один запрос для каждой из групп, создайте два датафрейма: test_results и control_results со столбцами time и avg_diff и только двумя строками.
times должно иметь значения before и after.
avg_diff содержит среднее значение дельты для всех пользователей за период времени до первого посещения ленты новостей каждым из них и после этого.
Учитываются только те пользователи, для которых имеются наблюдения и before, и after.

In [5]:
test_results=pd.io.sql.read_sql('''
select k.times, AVG(k.avg_diff) as avg_diff
from
(
                SELECT test.uid,"before" as times,AVG((strftime("%s", test.first_commit_ts)-deadlines.deadlines)/3600) as avg_diff
                FROM
                test LEFT JOIN deadlines ON test.labname = deadlines.labs
                WHERE test.labname not in ('project1') AND test.first_commit_ts < test.first_view_ts
                GROUP BY test.uid
                HAVING test.uid IN(SELECT DISTINCT uid FROM test WHERE labname not in ('project1') AND first_commit_ts > first_view_ts)

                union

                SELECT test.uid,"after" as times,avg((strftime("%s", test.first_commit_ts)-deadlines.deadlines)/3600) as avg_diff
                FROM test LEFT JOIN deadlines ON test.labname = deadlines.labs
                WHERE test.labname not in ('project1') AND test.first_commit_ts > test.first_view_ts
                GROUP BY test.uid
                HAVING test.uid IN(SELECT DISTINCT uid FROM test WHERE labname not in ('project1') AND first_commit_ts < first_view_ts)   
        ) k
        
        GROUP BY k.times                       

''',con)                
test_results

,times,avg_diff
0,after,-99.523810
1,before,-66.047619


In [6]:
control_results=pd.io.sql.read_sql('''
select k.times, AVG(k.avg_diff) as avg_diff
from
(
                SELECT control.uid,"before" as times,AVG((strftime("%s", control.first_commit_ts)-deadlines.deadlines)/3600) as avg_diff
                FROM
                control LEFT JOIN deadlines ON control.labname = deadlines.labs
                WHERE control.labname not in ('project1') AND control.first_commit_ts < control.first_view_ts
                GROUP BY control.uid
                HAVING control.uid IN(SELECT DISTINCT uid FROM control WHERE labname not in ('project1') AND first_commit_ts > first_view_ts)

                union

                SELECT control.uid,"after" as times,avg((strftime("%s", control.first_commit_ts)-deadlines.deadlines)/3600) as avg_diff
                FROM control LEFT JOIN deadlines ON control.labname = deadlines.labs
                WHERE control.labname not in ('project1') AND control.first_commit_ts > control.first_view_ts
                GROUP BY control.uid
                HAVING control.uid IN(SELECT DISTINCT uid FROM control WHERE labname not in ('project1') AND first_commit_ts < first_view_ts)   
        ) k
        
        GROUP BY k.times                       

''',con)                
control_results

,times,avg_diff
0,after,-99.322222
1,before,-98.033333


Закройте соединение.

In [ ]:
con.close()

Дайте ответ: оказалось ли предположение верным и влияет ли наличие страницы с новостной лентой на поведение учащихся?

Предположение верное, в тестовой группе значительная разница, а в контрольной нет

In [7]:
test_results

,times,avg_diff
0,after,-99.523810
1,before,-66.047619


In [8]:
control_results

,times,avg_diff
0,after,-99.322222
1,before,-98.033333
